In [1]:
import wandb
import torch
import yaml
from ecallisto_model import (
    ResNet,
)
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from tqdm import tqdm
from datasets import DatasetDict, load_dataset
import pandas as pd
from ecallisto_dataset import (
    EcallistoDatasetBinary,
    custom_resize,
    remove_background,
)


/home/vincenzo/miniconda3/envs/flaresense-v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ds_test = load_dataset("i4ds/radio-sunburst-ecallisto-paths-df-v2", split="test")

In [7]:
ds_test

Dataset({
    features: ['file_path', 'label', 'antenna', 'datetime'],
    num_rows: 30549
})

In [9]:
from sklearn.model_selection import train_test_split

# Access the dataset
data = ds_test['file_path']

# Extract labels
labels = ds_test['label']

# Perform stratified split
train_indices, val_indices = train_test_split(
    range(len(data)),
    test_size=0.2,        # Adjust the test_size as needed (e.g., 0.2 for 80/20 split)
    stratify=labels,      # Stratify based on labels to maintain label ratios
    random_state=42       # For reproducibility
)


In [11]:
from datasets import DatasetDict

# Create the training and validation datasets
train_dataset = ds_test.select(train_indices)
val_dataset = ds_test.select(val_indices)

# Create a DatasetDict with 'train' and 'validation' splits
split_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})


In [12]:
import pandas as pd

# Convert datasets to DataFrames for easy analysis
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()

# Calculate label distributions
train_label_counts = train_df['label'].value_counts(normalize=True)
val_label_counts = val_df['label'].value_counts(normalize=True)

print("Training Label Distribution:\n", train_label_counts)
print("\nValidation Label Distribution:\n", val_label_counts)


Training Label Distribution:
 label
False    0.865788
True     0.134212
Name: proportion, dtype: float64

Validation Label Distribution:
 label
False    0.865794
True     0.134206
Name: proportion, dtype: float64


In [13]:
# Define the new dataset name
new_dataset_name = 'i4ds/radio-sunburst-ecallisto-paths-df-v2-TEST-SET-SPLIT'

# Push to Hugging Face Hub
split_dataset.push_to_hub(new_dataset_name)


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/i4ds/radio-sunburst-ecallisto-paths-df-v2-TEST-SET-SPLIT/commit/9e8d67433b790254139be346d49129b8b81bd51d', commit_message='Upload dataset', commit_description='', oid='9e8d67433b790254139be346d49129b8b81bd51d', pr_url=None, pr_revision=None, pr_num=None)